In [1]:
import warnings
warnings.filterwarnings('ignore')

# About this notebook

* Author: Anubhav Jain
* Github repo: https://github.com/computron/pymatgen_tutorials
* YouTube video: https://youtu.be/b0tieiedGdg

![alt text](graphics/title.png "Title")

# Learn pymatgen
## Part 1: Structure and Molecule Fundamentals

This tutorial will cover:

* The ``Structure`` object
* The ``Molecule`` object
* Basic manipulation of ``Structure`` and ``Molecule`` objects
* Other examples and resources (pymatgen tests, LLMs, and matsci.org channel)

We will use the ``crystal_toolkit`` library to make the visualizations interactive. Let's start!

In [2]:
import crystal_toolkit  # for visualizing structures (install using ``pip install crystal-toolkit``)
import pymatgen  # pymatgen needs to be installed (v2024.6.10 at time of this tutorial)

# The Structure and Molecule objects

The ``Structure`` and ``Molecule`` objects are fundamental to most uses of pymatgen.

* A ``Structure`` refers to a periodic crystal
    * It has a lattice and basis
    * Examples include solids (metals, semiconductors, insulators, polymers), 2D surfaces, and liquids.
* A ``Molecule`` does not have periodic boundary conditions
    * No lattice, just a collection of atoms in space
    * Examples include individual molecules or small collections of molecules, nanoparticles, and molecules in implicit solvents.
    
After creating a ``Structure`` or ``Molecule``, you can analyze it, modify it, or use it to create input files for various simulation software.

![alt text](graphics/struct_mol.png "Structure and Molecule")

# Let's make a Structure object

We can create ``Structure`` objects:
* Using Python
* From a file
* From an API (eg. a database of structures like the Materials Project)


We'll start by creating a basic ``Structure`` using Python.

![alt text](graphics/python_file_api.png "Loading from Python, File, or API")

In [3]:
from pymatgen.core import Lattice, Structure

# Define the lattice (cell lengths and angles). Note that the units are Angstroms
a, b, c = 3.9, 3.9, 3.9
alpha, beta, gamma = 90, 90, 90
lattice = Lattice.from_parameters(a, b, c, alpha, beta, gamma)

# Define the basis (atomic positions and species)
species = ["Pb", "Ti", "O", "O", "O"]
coords = [
    [0.0, 0.0, 0.0],         # Pb
    [0.5, 0.5, 0.5],         # Ti
    [0.5, 0.5, 0.0],         # O1
    [0.0, 0.5, 0.5],         # O2
    [0.5, 0.0, 0.5]          # O3
]

# Create the structure
PTO_structure = Structure(lattice, species, coords)
print(PTO_structure)  # print the Structure
PTO_structure # visualize the structure using crystal_toolkit

Full Formula (Ti1 Pb1 O3)
Reduced Formula: TiPbO3
abc   :   3.900000   3.900000   3.900000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (5)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Pb    0    0    0
  1  Ti    0.5  0.5  0.5
  2  O     0.5  0.5  0
  3  O     0    0.5  0.5
  4  O     0.5  0    0.5


# What can you do directly with a Structure?

* Once you define a ``Structure``, many different functions and analyses are available
* Some functions are implicitly defined via *inheritance*
* The inheritance diagram below illustrates how the ``Structure`` object *inherits* attributes and functions of its parent classes.
* Using an IDE software can help make it much clearer what's available.


![alt text](graphics/inheritance.png "Inheritance Diagrams for Structure and Molecule")

In [4]:
# Some examples of things you can access for a Structure are below, but there are many more!
print(PTO_structure.composition)  # This is a Composition object
print(PTO_structure.composition.anonymized_formula)  # Composition objects themselves have many attributes/methods
print(PTO_structure.density)
print(PTO_structure[0])  # this is the same as PTO_structure.sites[0]
print(PTO_structure[1])
print(PTO_structure[1].frac_coords)  # fractional vs cartesian coordinates
print(PTO_structure.get_distance(0, 1))  # distance between sites 0 and 1
print(PTO_structure.get_neighbors(PTO_structure[1], 3))  # get all neighbors within 3 Angstroms of site 1 (Ti)

Pb1 Ti1 O3
ABC3
8.483818073921379 g cm^-3
[0. 0. 0.] Pb
[1.95 1.95 1.95] Ti
[0.5 0.5 0.5]
3.37749907475931
[PeriodicNeighbor: O (-1.194e-16, 1.95, 1.95) [0.0, 0.5, 0.5], PeriodicNeighbor: O (1.95, 0.0, 1.95) [0.5, 0.0, 0.5], PeriodicNeighbor: O (1.95, 1.95, 2.388e-16) [0.5, 0.5, 0.0], PeriodicNeighbor: O (3.9, 1.95, 1.95) [1.0, 0.5, 0.5], PeriodicNeighbor: O (1.95, 3.9, 1.95) [0.5, 1.0, 0.5], PeriodicNeighbor: O (1.95, 1.95, 3.9) [0.5, 0.5, 1.0]]


# Understanding the Sites within a Structure

* In pymatgen, the ``Site`` object represents a physical point in space
    * For ``Structure`` objects, which have periodic boundary conditions, this is technically a ``PeriodicSite``.
* The ``Site`` object contains a ``species`` attribute that represents an atom or other entity at that point in space.
* The ``species`` attribute is complex because it can be one of many many things:
    * Simple structures that just have an atom on a site: easiest to just use the ``specie`` field, which contains an ``Element`` object
    * ``Structure`` objects where ions are present on a site: still use the ``specie`` field, but this now contains a ``Specie`` object
    * Even more complex elements that have partial occupancies of sites must use the ``species`` field (note this is now plural)

![alt text](graphics/site_diagram.png "Diagram showing Structure, Site, and Species")

In [5]:
print(PTO_structure[0].specie)  # note that we are using the shortcut "specie"
print(type(PTO_structure[0].specie))  # the type of specie is an Element
print(PTO_structure[0].specie.Z)  # Element objects have attributes
print(PTO_structure[0].specie.common_oxidation_states)
print(PTO_structure[0].specie.atomic_mass)

Pb
<enum 'Element'>
82
(2, 4)
207.2 amu


# Modifying Structures

* Next, let's modify ``Structure`` objects
* Basic modifications can be done with methods of the ``Structure`` object
    * There are much more advanced transformations defined in pymatgen's ``transformations`` package
* After modifying structures, you can export them to various formats (e.g., CIF, POSCAR).

* Note: pymatgen also contains an ``IStructure`` object, which is an **immutable** version of a ``Structure``. This object is helpful if you:
    * want to use a Structure as a default argument in a function header (common Python pitfall)
    * use Structures as keys in dictionaries or iterating over lists
    * want to adopt a functional programming style

![alt text](graphics/structure_mod.png "Modifying structures")

In [6]:
PTO_structure.copy().make_supercell([2, 3, 1])  # note use of copy() to avoid overwriting original

In [7]:
PTO_structure.copy().apply_strain([0.5, 0.2, 0])

In [8]:
PTO_structure.copy().replace(4, "S")  # site 4 is an oxygen atom

In [9]:
PTO_structure.copy().replace_species({"O":"S", "Pb": "Sn"})

In [10]:
PTO_structure.copy().remove_species(["Pb"])

In [11]:
PTO_structure.copy().insert(-1, "H", [0.25, 0.25, 0.25])

In [12]:
# Chaining structure modifications
my_new_structure = PTO_structure.copy()
my_new_structure.replace_species({"O":"S", "Pb": "Sn"})  # note we are modifying in-place and no longer copying
my_new_structure.insert(-1, "H", [0.25, 0.25, 0.25])
my_new_structure.apply_strain([0.5, 0.2, 0])
my_new_structure.make_supercell([2, 3, 1])

In [13]:
# Analyze the new structure
print(my_new_structure.composition)
print(my_new_structure.composition.reduced_formula)
print(my_new_structure.density)

Sn6 Ti6 S18 H6
TiSnHS3
4.102272790200292 g cm^-3


In [14]:
# Export a Structure to a VASP POSCAR format
my_new_structure.to('POSCAR', fmt="POSCAR") # this writes a file in addition to returning the raw string

# Advanced note: You can use pymatgen's POSCAR class for more options
# You can also enter kwargs intended for the POSCAR class directy to the above function call

'Ti6 Sn6 H6 S18\n1.0\n  11.6999999999999993    0.0000000000000000    0.0000000000000007\n  -0.0000000000000009   14.0399999999999991    0.0000000000000009\n   0.0000000000000000    0.0000000000000000    3.8999999999999999\nSn Ti S H S\n6 6 12 6 6\ndirect\n   0.0000000000000000    0.0000000000000000    0.0000000000000000 Sn\n   0.0000000000000000    0.3333333333333333    0.0000000000000000 Sn\n   0.0000000000000000    0.6666666666666666    0.0000000000000000 Sn\n   0.5000000000000000    0.0000000000000000    0.0000000000000000 Sn\n   0.5000000000000000    0.3333333333333333    0.0000000000000000 Sn\n   0.4999999999999999    0.6666666666666666    0.0000000000000000 Sn\n   0.2500000000000000    0.1666666666666667    0.5000000000000000 Ti\n   0.2500000000000000    0.5000000000000000    0.4999999999999999 Ti\n   0.2500000000000000    0.8333333333333333    0.5000000000000001 Ti\n   0.7499999999999999    0.1666666666666667    0.5000000000000000 Ti\n   0.7499999999999999    0.5000000000000000 

In [15]:
# Export a Structure to Crystallographic Information File/Framework (CIF) format
my_new_structure.to('output.cif', fmt="cif")  # this writes a file in addition to returning the raw string

# Advanced note: You can use pymatgen's CIFWriter class for more options
# You can also enter kwargs intended for the CIFWriter class directy to the above function call

"# generated using pymatgen\ndata_TiSnHS3\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   11.70000000\n_cell_length_b   14.04000000\n_cell_length_c   3.90000000\n_cell_angle_alpha   90.00000000\n_cell_angle_beta   90.00000000\n_cell_angle_gamma   90.00000000\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   TiSnHS3\n_chemical_formula_sum   'Ti6 Sn6 H6 S18'\n_cell_volume   640.64520000\n_cell_formula_units_Z   6\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Sn  Sn0  1  0.00000000  0.00000000  0.00000000  1.0\n  Sn  Sn1  1  0.00000000  0.33333333  0.00000000  1.0\n  Sn  Sn2  1  0.00000000  0.66666667  0.00000000  1.0\n  Sn  Sn3  1  0.50000000  0.00000000  0.00000000  1.0\n  Sn  Sn4  1  0.50000000  0.33333333  0.00000000  1.0\n  Sn  Sn5  1  0.50000000  0.66

# Loading a Structure from a file

* Let's load a ``Structure`` from a file in CIF format
* This ``Structure`` will contain ions on the sites instead of elements

![alt text](graphics/python_file_api.png "Loading from Python, File, or API")

In [16]:
# Load the structure from a file (e.g., CIF, POSCAR, CSSR)
PTO_structure_from_cif = Structure.from_file("files/PbTiO3.cif")
print(PTO_structure_from_cif)  # print the structure - note the oxidation states!
PTO_structure_from_cif

Full Formula (Ti1 Pb1 O3)
Reduced Formula: TiPbO3
abc   :   3.879552   3.879552   4.285888
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (5)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Ti4+  0.5  0.5  0.582514
  1  Pb2+  0    0    0.123588
  2  O2-   0    0.5  0.494158
  3  O2-   0.5  0    0.494158
  4  O2-   0.5  0.5  0.995782


# Using specie for Structures with ions on sites rather than elements

* Now, when we use the ``specie`` shortcut on a ``Site``, we will get a ``Species`` object which contains some additional information such as oxidation state

![alt text](graphics/site_diagram.png "Diagram showing Structure, Site, and Species")

In [17]:
print(PTO_structure_from_cif.sites[0].specie)  # this has an Element and oxidation state
print(type(PTO_structure_from_cif.sites[0].specie))  # this is of type Species
print(PTO_structure_from_cif.sites[0].specie.oxi_state)
print(PTO_structure_from_cif.sites[0].specie.ionic_radius)
print(PTO_structure_from_cif.sites[0].specie.element)  # you can also get the Element

Ti4+
<class 'pymatgen.core.periodic_table.Species'>
4.0
0.745 ang
Ti


# Loading a Structure from a file

* Let's load another ``Structure`` from a file in CIF format
* This ``Structure`` will contain compositions on the sites instead of elements

![alt text](graphics/python_file_api.png "Loading from Python, File, or API")

In [18]:
# Load the structure from a file (e.g., CIF, POSCAR, CSSR)
SrYCO_structure = Structure.from_file("files/SrYCoO.cif")
print(SrYCO_structure)  # print the Structure - note the partial occupancies!
SrYCO_structure

Full Formula (Sr11.2 Y4.8 Co16 O42)
Reduced Formula: Sr11.2Y4.8Co16O42
abc   :   7.623900   7.623900  15.327000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (80)
  #  SP                  a       b       c
---  -------------  ------  ------  ------
  0  Sr:0.7, Y:0.3  0       0       0.8785
  1  Sr:0.7, Y:0.3  0.5     0.5     0.3785
  2  Sr:0.7, Y:0.3  0       0       0.1215
  3  Sr:0.7, Y:0.3  0.5     0.5     0.6215
  4  Sr:0.7, Y:0.3  0       0.5     0.867
  5  Sr:0.7, Y:0.3  0.5     0       0.367
  6  Sr:0.7, Y:0.3  0       0.5     0.133
  7  Sr:0.7, Y:0.3  0.5     0       0.633
  8  Sr:0.7, Y:0.3  0.5     0       0.867
  9  Sr:0.7, Y:0.3  0       0.5     0.367
 10  Sr:0.7, Y:0.3  0.5     0       0.133
 11  Sr:0.7, Y:0.3  0       0.5     0.633
 12  Sr:0.7, Y:0.3  0       0       0.3503
 13  Sr:0.7, Y:0.3  0.5     0.5     0.8503
 14  Sr:0.7, Y:0.3  0       0       0.6497
 15  Sr:0.7, Y:0.3  0.5     0.5     0.1497
 16  Co             0.7483  0

# Structures with partial occupancies (multiple species on a site)

* Some structures may gave **partial occupancy** or **disorder** on sites.
    * Example: a site is sometimes occupied by Li, sometimes by Na (in a 50:50 ratio)
    * Example: a site is sometimes occupied by Li, sometimes is vacant altogether (in a 75:25 ratio)
* Structure with partial occupancies will have a ``Composition`` object in the ``species`` key.
    * Note that calling the ``specie`` key (singular form) on such structures will result in an error!

![alt text](graphics/site_diagram.png "Diagram showing Structure, Site, and Species")

In [19]:
print(SrYCO_structure.is_ordered)
print(SrYCO_structure.sites[0].is_ordered)
print(SrYCO_structure.sites[20].is_ordered)
print(SrYCO_structure.sites[0].species)  # note that this is "species", not "specie"!
print(type(SrYCO_structure.sites[0].species))
print(SrYCO_structure.sites[0].species["Sr"])
print(SrYCO_structure.sites[0].species["Y"])

False
False
True
Sr0.7 Y0.3
<class 'pymatgen.core.composition.Composition'>
0.7
0.3


# Loading a structure from an API (Materials Project)

* Finally, we will load a crystal structure from an online database (The Materials Project)
* Using the API will give you instant access to >150,000 crystal structures that are available on the Materials Project web site
* This structure will have magnetic moments defined

![alt text](graphics/python_file_api.png "Loading from Python, File, or API")

In [20]:
from pymatgen.ext.matproj import MPRester

API_KEY = "YOUR_API_KEY" # Replace with your API key from www.materialsproject.org

# Create an MPRester object
mpr = MPRester(API_KEY)

# Fetch the structure using the Materials Project ID
mp_id = "mp-1069079"  # this is the Materials Project ID
BFO_structure = mpr.get_structure_by_material_id(mp_id)

print(BFO_structure)  # print the Structure - notice the "magmom"
BFO_structure  # visualize

Retrieving MaterialsDoc documents: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 1686.49it/s]


Full Formula (Fe2 Bi2 O6)
Reduced Formula: FeBiO3
abc   :   4.879828   5.305300   5.305304
angles:  90.000015  90.000019  90.000019
pbc   :       True       True       True
Sites (10)
  #  SP           a         b         c    magmom
---  ----  --------  --------  --------  --------
  0  Fe    0.476167  0         0.500001    -4.23
  1  Fe    0.476163  0.5       0            4.23
  2  Bi    0.048707  0         0           -0
  3  Bi    0.048707  0.5       0.499999    -0
  4  O     0.856306  0.5       0            0.185
  5  O     0.856303  0         0.5         -0.185
  6  O     0.330413  0.249999  0.250001     0
  7  O     0.330412  0.750001  0.749999     0
  8  O     0.330412  0.249999  0.749998     0
  9  O     0.330412  0.750002  0.250001     0


# Understanding site properties

* We can use the ``properties`` attribute of a ``Site`` to define properties like magnetic moment
* Defining such properties can be useful because:
    * They are used when creating various inputs for simulation software, e.g., the MAGMOM tag in the VASP software
    * They can be used for your own internal analysis or for visualization purposes
* Note: it can be confusing about what goes in ``Site.properties`` vs ``Species.properties``. In general, pymatgen recommends that users self-define properties in ``Site.properties``, with ``Species.properties`` reserved for the ``spin`` that is parsed from simulation outputs.
    * As a user, the general guidance is to define things using ``Site.properties``.

![alt text](graphics/site_diagram.png "Diagram showing Structure, Site, and Species")

In [21]:
BFO_structure.sites[0].properties

{'magmom': -4.23}

# The Molecule object

* The ``Molecule`` object is used for systems without periodic boundary conditions, e.g., isolated molecules
* A ``Molecule`` is in some ways simpler than a ``Structure`` because we no longer have a lattice

![alt text](graphics/struct_mol.png "Structure and Molecule")

In [22]:
from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.analysis.local_env import CovalentBondNN
from pymatgen.core.structure import Molecule

# Define the atomic species and their coordinates for acetone (C3H6O)
species = ["H", "C", "H", "H", "C", "O", "C", "H", "H", "H"]
coords = [
    [2.0471, 0.1029, 0.7251],   # H
    [1.4376, 0.0818, -0.1875],  # C
    [1.6643, 0.9894, -0.7620],  # H
    [1.7616, -0.7796, -0.7860], # H
    [-0.0377, -0.0035, 0.0968], # C
    [-0.4797, -0.0446, 1.2294], # O
    [-0.9294, -0.0345, -1.1150],# C
    [-0.7844, 0.8703, -1.7195], # H
    [-0.6856, -0.8986, -1.7467],# H
    [-1.9948, -0.0979, -0.8587] # H
]

# Create the molecule object
molecule = Molecule(species, coords)
print(molecule)
molecule

Full Formula (H6 C3 O1)
Reduced Formula: H6C3O
Charge = 0.0, Spin Mult = 1
Sites (10)
0 H     2.047100     0.102900     0.725100
1 C     1.437600     0.081800    -0.187500
2 H     1.664300     0.989400    -0.762000
3 H     1.761600    -0.779600    -0.786000
4 C    -0.037700    -0.003500     0.096800
5 O    -0.479700    -0.044600     1.229400
6 C    -0.929400    -0.034500    -1.115000
7 H    -0.784400     0.870300    -1.719500
8 H    -0.685600    -0.898600    -1.746700
9 H    -1.994800    -0.097900    -0.858700


# Inheritance in molecules

* Like ``Structure``, the ``Molecule`` inherits many of its capabilities from its parent classes

![alt text](graphics/inheritance.png "Inheritance Diagrams for Structure and Molecule")

In [23]:
print(molecule.composition)
print(molecule.composition.iupac_formula)
print(molecule[0])  # Site object, shortcut to molecule.sites[0]
print(molecule[0].specie)  # Element object - remember that specifying "species" will yield a Composition

H6 C3 O1
C3 H6 O1
[2.0471 0.1029 0.7251] H
H


# Modifying molecules

* One can do basic modifications of molecules (including functional group substitutions) with methods of the ``Molecule`` object itself
* After modifying molecules, you can export the result to various formats (e.g., xyz, gm03, etc).

* Note: pymatgen also contains an ``IMolecule`` object, which is an **immutable** version of a ``Molecule``. This object is helpful if you:
    * want to use a molecule as a default argument in a function header (common Python pitfall)
    * use molecule as keys in dictionaries or iterating over lists
    * want to adopt a functional programming style


In [24]:
# Let's add a functional group
subs_mol = molecule.copy().substitute(3, "methyl")

subs_mol

[2024-09-12 15:25:41,815] ERROR in app: Exception on /_dash-component-suites/dash/dash_table/async-table.js [GET]
Traceback (most recent call last):
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-

In [25]:
# Let's add a functional group
# Define the functional group to substitute (methyl group - CH3)
sub = Molecule(["X", "C", "H", "H", "H"], [
    [0.000000, 0.000000, 1.08],
    [0.000000, 0.000000, 0.000000],
    [1.026719, 0.000000, -0.363000],
    [-0.513360, -0.889165, -0.363000],
    [-0.513360, 0.889165, -0.363000],
])

subs_mol = molecule.copy().substitute(3, sub)
subs_mol



[2024-09-12 15:25:43,731] ERROR in app: Exception on /_dash-layout [GET]
Traceback (most recent call last):
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^

[2024-09-12 15:25:44,246] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ajain/Documents/code_venvs/pmg_tutorials/lib/python3.11/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^

In [26]:
# Export a Molecule to xyz file format
molecule.to('output.xyz', fmt="xyz")  # this writes a file in addition to returning the raw string

# Advanced note - if you install the OpenBabel pymatgen adapter, you can access many more file formats

'10\nH6 C3 O1\nH 2.047100 0.102900 0.725100\nC 1.437600 0.081800 -0.187500\nH 1.664300 0.989400 -0.762000\nH 1.761600 -0.779600 -0.786000\nC -0.037700 -0.003500 0.096800\nO -0.479700 -0.044600 1.229400\nC -0.929400 -0.034500 -1.115000\nH -0.784400 0.870300 -1.719500\nH -0.685600 -0.898600 -1.746700\nH -1.994800 -0.097900 -0.858700'

In [27]:
# Let's add some bonds

# Create a CovalentBondNN object
nn = CovalentBondNN()

# Generate the MoleculeGraph
mol_graph = MoleculeGraph.from_local_env_strategy(molecule, nn)

mol_graph

# More code examples and getting help

* Congratulations, you now know the fundamentals of the ``Structure`` and ``Molecule`` objects!
* This is the foundation of most pymatgen usage, and understanding these objects will go long way towards learning other pymatgen capabilities
* Often, users want additional code examples or need additional help. The next slides show additional ways to do so.



# Pymatgen unit tests as example code

* A "hidden" set of pymatgen usage examples are in the unit tests of pymatgen, which are present if you download pymatgen from Github
* Because pymatgen has extensive coverage of its functions in unit tests, chances are that you can find a usage example of your desired function within the unit tests

![alt text](graphics/pmg_tests.png "Example of worked code examples in pymatgen unit tests")


# Large Language Models

* Large language models like ChatGPT and Github Copilot are often capable of generating correct pymatgen syntax
* However, the outputs of these tools should still be checked as they still can produce incorrect code

![alt text](graphics/chatgpt.png "ChatGPT help example")

# Matsci.org help forum

* There is also a help channel dedicated to pymatgen at www.matsci.org

![alt text](graphics/matsci_discourse.png "Screenshot of pymatgen matsci.org page")
